# 🕵️ CSI de los Datos: Detectives del Fraude (Pandas + IA)
**Bootcamp:** IA Innovador — Laboratorio guiado (3 h)  
**By:** Ing. Engler González

**Objetivo:** Investigar patrones sospechosos en transacciones usando **pandas** y (opcionalmente) **Gemini** para redactar una narrativa ejecutiva.

> **Nota:** El lab funciona 100% con *pandas* aunque no configures la IA.  
> Para usar IA, crea la variable de entorno `GOOGLE_API_KEY` en Colab: *Entorno de ejecución → Configurar variables de entorno → Añadir*.


## 0) Setup

In [ ]:
!pip -q install pandas numpy matplotlib google-generativeai

import os, math, random, json, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import google.generativeai as genai
from IPython.display import display

np.random.seed(7); pd.set_option("display.max_colwidth", 120)
# Retrieve the API key from Colab secrets
try:
    from google.colab import userdata
    api_key = userdata.get("GOOGLE_API_KEY")
except ImportError:
    # Fallback for environments where userdata is not available
    api_key = os.environ.get("GOOGLE_API_KEY")

model = None
# (Opcional IA) Configura tu GOOGLE_API_KEY en Entorno de ejecución > Variables de entorno
if api_key:
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel("gemini-1.5-flash")
    print("✅ Gemini model configured.")
else:
    print("❌ GOOGLE_API_KEY not found. Gemini model not configured.")

✅ Gemini model configured.


## 1) Generar dataset sintético
No necesitas archivos. Simularemos **60 días** de transacciones en múltiples países y canales. Inyectaremos anomalías a propósito (madrugada, montos altos, duplicados, valores faltantes).


In [ ]:
# Escenario: transacciones de 60 días. Inyectamos anomalías.
n = 6000
countries = ["CO", "MX", "US", "AR", "CL", "ES"]
channels = ["web", "app", "social_ads", "affiliate", "email"]

# Distribución de horas por tramos (más peso en el día). Normalizamos para que sume 1.
hour_weights = [0.02]*6 + [0.04]*6 + [0.06]*6 + [0.02]*6  # 0-5, 6-11, 12-17, 18-23
hour_weights = np.array(hour_weights, dtype=float)
hour_weights = hour_weights / hour_weights.sum()

hours = np.random.choice(range(0,24), size=n, p=hour_weights)
base_amount = np.random.lognormal(mean=3.3, sigma=0.5, size=n) * 10  # distribución sesgada
country = np.random.choice(countries, size=n, p=[0.32,0.20,0.16,0.12,0.12,0.08])
channel = np.random.choice(channels, size=n, p=[0.35,0.25,0.15,0.15,0.10])
user_id = np.random.randint(1000, 5000, size=n)
days = pd.date_range(end=pd.Timestamp.today().normalize(), periods=60)
timestamp = np.random.choice(days, size=n) + pd.to_timedelta(hours, unit="h")

# Inyectar patrones sospechosos
amount = base_amount.copy()
mask_night = (hours < 6) | (hours > 22)
amount[mask_night] *= np.random.uniform(1.4, 2.1, size=mask_night.sum())  # montos más altos de madrugada

mask_remote = np.isin(country, ["ES","US"]) & np.isin(channel, ["affiliate","social_ads"])
amount[mask_remote] *= np.random.uniform(1.2, 1.8, size=mask_remote.sum())

# Outliers y duplicados
idx_out = np.random.choice(range(n), size=40, replace=False)
amount[idx_out] *= np.random.uniform(3, 10, size=40)
duplicated_rows = 30
dups = np.random.choice(range(n), size=duplicated_rows, replace=False)

df = pd.DataFrame({
    "transaction_id": [f"T{100000+i}" for i in range(n)],
    "user_id": user_id,
    "amount": amount.round(2),
    "country": country,
    "channel": channel,
    "timestamp": timestamp
})
df = pd.concat([df, df.iloc[dups]], ignore_index=True)  # añadir duplicados

# Nulos/errores de formato
df.loc[np.random.choice(df.index, 50, replace=False), "channel"] = None
df.loc[np.random.choice(df.index, 30, replace=False), "country"] = "??"
df.loc[np.random.choice(df.index, 20, replace=False), "amount"] = None

print("✅ Dataset generado:", df.shape)
df.head()

## 2) Limpieza de datos (GUIADA)
**TODOs**  
1. Revisar info general, nulos y duplicados.  
2. Eliminar duplicados exactos.  
3. Asegurar tipos correctos (`timestamp` → datetime, `amount` → float).  
4. Normalizar categorías (reemplazar `"??"` por `NaN`, decidir imputación/filtrado).  
5. Tratar nulos en `amount` y `channel` (documentar la decisión).


In [ ]:
---# Punto de partida y pistas ---
df.info()
print("\nNulos antes:", df.isna().sum())
df.isna().mean().sort_values(ascending=False).head(10)

print("\nDuplicados antes:", df.duplicated().sum())
df = df.drop_duplicates()
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df["amount"] = pd.to_numeric(df["amount"], errors="coerce")
df["country"] = df["country"].replace("??", np.nan)

Treat nulls in 'amount' and 'channel'
For 'amount', we can fill with the median as it's less sensitive to outliers than the mean.
df["amount"] = df["amount"].fillna(df["amount"].median())
For 'channel', we can fill with a placeholder like 'unknown' as it's a categorical variable.
df["channel"] = df["channel"].fillna("unknown")

print("\nNulos después:", df.isna().sum())

NameError: name 'df' is not defined

## 3) EDA: entender el comportamiento
**TODOs**  
6. Crear columnas derivadas: `hour`, `day_of_week`.  
7. Resúmenes clave:  
   - monto promedio por país  
   - distribución por canal  
   - actividad por hora  
   - top usuarios por monto total (opcional)


In [ ]:
df["hour"] = df["timestamp"].dt.hour
df["day_of_week"] = df["timestamp"].dt.day_name()

mean_by_country = df.groupby("country", dropna=True)["amount"].mean().sort_values(ascending=False)
count_by_channel = df["channel"].value_counts(dropna=False)
activity_by_hour = df.groupby("hour")["transaction_id"].count()

print("Monto promedio por país:"); display(mean_by_country.head(10))
print("\nDistribución por canal:"); display(count_by_channel)
print("\nActividad por hora:"); display(activity_by_hour.head(24))

# Gráfico simple (matplotlib)
plt.figure()
activity_by_hour.plot(kind="bar")
plt.title("Transacciones por hora")
plt.xlabel("Hora del día")
plt.ylabel("Conteo")
plt.show()

## 4) Detección de outliers y reglas de sospecha
**TODOs**  
8. Detectar outliers por IQR en `amount`.  
9. Crear reglas heurísticas (ajusta umbrales con tu EDA):  
   - madrugada (`hour` < 6 o > 22) con `amount` > p95  
   - `affiliate`/`social_ads` con `amount` > p95  
   - `country` o `channel` nulos con `amount` > p95  
10. Construir un **score de sospecha** sumando reglas (0–3/4).


In [ ]:
q1, q3 = df["amount"].quantile([0.25, 0.75])
iqr = q3 - q1
lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
df["is_outlier_amount"] = (df["amount"] < lower) | (df["amount"] > upper)

p95 = df["amount"].quantile(0.95)
df["is_night"] = (df["hour"] < 6) | (df["hour"] > 22)
df["rule_night_high"] = df["is_night"] & (df["amount"] > p95)
df["rule_affiliate_high"] = df["channel"].isin(["affiliate","social_ads"]) & (df["amount"] > p95)
df["rule_missing_high"] = (df["country"].isna() | df["channel"].isna()) & (df["amount"] > p95)

rule_cols = ["is_outlier_amount","rule_night_high","rule_affiliate_high","rule_missing_high"]
df["suspicion_score"] = df[rule_cols].sum(axis=1)

df[["amount","hour","country","channel","is_outlier_amount","rule_night_high","rule_affiliate_high","rule_missing_high","suspicion_score"]].head(10)

## 5) Ranking de casos y explicación
**TODOs**  
11. Ordena por `suspicion_score` y revisa el **Top 20**. Escribe 3 observaciones.


In [ ]:
top_cases = df.sort_values(["suspicion_score","amount"], ascending=[False, False]).head(20)
top_cases[["transaction_id","user_id","amount","country","channel","hour","suspicion_score"]]

## 6) (Opcional) IA para narrativa ejecutiva
Si tienes `GOOGLE_API_KEY`, genera un *brief* de 6 líneas para directivos con patrones, acciones y métricas.


In [ ]:
if model:
    resumen_stats = {
        "p95_amount": float(p95),
        "outlier_rate": float(df["is_outlier_amount"].mean()),
        "night_rate": float(df["is_night"].mean()),
        "top_channels": df["channel"].value_counts().head(3).to_dict()
    }
    prompt = f"""
    Eres analista forense de datos. Con base en:
    Stats: {json.dumps(resumen_stats)}
    Reglas aplicadas: {rule_cols}
    Redacta un briefing ejecutivo (6 líneas) explicando:
    - Qué patrones sugieren posible fraude
    - Acciones inmediatas (reglas de negocio, límites, monitoreo)
    - Métricas a vigilar la próxima semana
    """
    print(model.generate_content(prompt).text)
else:
    print("⚠️ IA no configurada. El análisis se puede entregar igual con pandas y gráficos.")

## 7) Reto final (entrega)
12. Ajusta y compara umbrales (`p90`/`p95`/`p99`).  
13. Añade más países y **1 regla nueva** por país-hora.  
14. Genera **2 gráficos** que respalden tus reglas (ej: día vs noche, por canal).  
15. Redacta **5 conclusiones numeradas**.


In [ ]:
print("🧩 Reto final: ajusta umbrales, agrega 1 regla nueva, 2 gráficos y 5 conclusiones.")

In [ ]:
# --- Punto de partida y pistas ---
df.info()
print("\nNulos antes:", df.isna().sum())
df.isna().mean().sort_values(ascending=False).head(10)

print("\nDuplicados antes:", df.duplicated().sum())
df = df.drop_duplicates()
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df["amount"] = pd.to_numeric(df["amount"], errors="coerce")
df["country"] = df["country"].replace("??", np.nan)

# Treat nulls in 'amount' and 'channel'
# For 'amount', we can fill with the median as it's less sensitive to outliers than the mean.
df["amount"] = df["amount"].fillna(df["amount"].median())
# For 'channel', we can fill with a placeholder like 'unknown' as it's a categorical variable.
df["channel"] = df["channel"].fillna("unknown")

print("\nNulos después:", df.isna().sum())

NameError: name 'df' is not defined